Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [9]:

library("ISLR2")
library("tidymodels")


Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.9     ✔ recipes      1.3.1
✔ dials        1.4.1     ✔ rsample      1.3.1
✔ dplyr        1.1.4     ✔ tibble       3.3.0
✔ ggplot2      3.5.2     ✔ tidyr        1.3.1
✔ infer        1.0.9     ✔ tune         1.3.0
✔ modeldata    1.5.0     ✔ workflows    1.2.0
✔ parsnip      1.3.2     ✔ workflowsets 1.1.1
✔ purrr        1.1.0     ✔ yardstick    1.3.2

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [2]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [3]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [12]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [10]:
mod = logistic_reg()  |> set_engine("glm") |> set_mode("classification") 

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [13]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9658889


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [16]:
folds = vfold_cv(Default_train, v = 10, strata = default)
folds |> glimpse()



Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [17]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97411111,10,0.0009377029,Preprocessor1_Model1
brier_class,binary,0.02127807,10,0.0008939037,Preprocessor1_Model1
roc_auc,binary,0.94850487,10,0.0057805045,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

The accuracy of the model is 97.41% which is slightly higher than the 96.58% with the naive model


Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [21]:
rec = recipe(default ~ income + balance + student, data = Default_train) |>
    step_dummy(student)

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97377778,10,0.0009543777,Preprocessor1_Model1
brier_class,binary,0.02119569,10,0.0008130705,Preprocessor1_Model1
roc_auc,binary,0.94907423,10,0.0055581107,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

The model has a slightly lower mean accuracy, so no it does not appear that the model improves with student as a predictor



Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [22]:
rec = recipe(default ~ income + balance, data = Default_test)

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()



.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97411111,10,0.0009377029,Preprocessor1_Model1
brier_class,binary,0.02127807,10,0.0008939037,Preprocessor1_Model1
roc_auc,binary,0.94850487,10,0.0057805045,Preprocessor1_Model1



Yes, the accuracy of our model is 97.4% which is higher than the naive model
